## 0.0 Imports

In [2]:
import pandas as pd
import inflection
import math

0.1 Helper Functioons

## 0.2 Loading data

In [3]:
df_sales_raw = pd.read_csv('./data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('./data/store.csv', low_memory=False)

#df_raw = pd.concat([df_sales_raw, df_store_raw]) - juntando dois datasets

df_raw = pd.merge(df_sales_raw, df_store_raw, how = 'left', on = 'Store')

In [4]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
259735,1090,7,2014-12-07,0,0,0,0,0,0,a,a,330.0,NaN,NaN,1,14.0,2011.0,"Jan,Apr,Jul,Oct"


## 1.0 Descrição dos dados

In [5]:
df1 = df_raw.copy()

1.1 Rename Columns

In [6]:
df1.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval'],
      dtype='object')

In [7]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']

# rename dados da lista de kamokase para snakecase


snakecase = lambda x: inflection.underscore(x)

cols_new = list(map ( snakecase, cols_old ) )

df1.columns = cols_new

In [8]:
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Dimensão do Dataset

In [9]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Cols: {}'.format(df1.shape[1]))

Number of Rows: 1017209
Number of Cols: 18


## 1.3 Tipo de dados

In [10]:
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Check NA

In [11]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

## 1.5 Limpando os NA

In [31]:
#competition_distance
df1['competition_distance'] = df1['competition_distance'].apply(lambda x: 200000.0 if math.isnan(x) else x)


#competition_open_since_month
df1['competition_open_since_month'] = df1.apply(lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x ['competition_open_since_month'], axis=1)

#competition_open_year
df1['competition_open_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan (x['competition_open_since_year']) else x ['competition_open_since_year'], axis=1 )


#promo2_since_week
df1['promo2_since_week'] = df1.apply(lambda x: x['date'].week if math.isnan (x['promo2_since_week']) else x['promo2_since_week'], axis=1)


#promo2_since_year
df1['promo2_since_year'] = df1.apply(lambda x: x['date'].year if math.isnan (x['promo2_since_year']) else x['promo2_since_year'], axis=1)


#promo_interval
month_map = {1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'} 

df1['promo_interval'].fillna(0, inplace=True)

df1['month_map'] = df1['date'].dt.month.map(month_map)


df1['is_promo'] = df1[['promo_interval', 'month_map']].apply(lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)


In [32]:
df1.sample(15).T

,694278,849239,983747,317269,377018,600151,125106,271220,515618,782639,722368,909131,909178,451264,796837
store,419,395,1103,572,462,1067,227,301,159,695,634,77,124,475,398
day_of_week,4,5,4,1,7,5,5,1,3,2,7,7,7,5,3
date,2013-10-17 00:00:00,2013-05-31 00:00:00,2013-01-31 00:00:00,2014-10-06 00:00:00,2014-08-03 00:00:00,2014-01-10 00:00:00,2015-04-10 00:00:00,2014-11-24 00:00:00,2014-03-26 00:00:00,2013-07-30 00:00:00,2013-09-22 00:00:00,2013-04-07 00:00:00,2013-04-07 00:00:00,2014-05-23 00:00:00,2013-07-17 00:00:00
sales,3549,7138,3690,8331,0,6697,5692,7762,5561,8614,0,0,0,5915,5585
customers,708,807,299,639,0,550,564,724,544,739,0,0,0,941,596
open,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1
promo,0,1,0,1,0,1,0,1,0,1,0,0,0,1,1
state_holiday,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
school_holiday,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1
store_type,c,a,d,d,a,d,a,a,d,a,d,d,a,a,c


In [30]:
df1.isna().sum()

store                           0
day_of_week                     0
date                            0
sales                           0
customers                       0
open                            0
promo                           0
state_holiday                   0
school_holiday                  0
store_type                      0
assortment                      0
competition_distance            0
competition_open_since_month    0
competition_open_since_year     0
promo2                          0
promo2_since_week               0
promo2_since_year               0
promo_interval                  0
month_map                       0
is_promo                        0
dtype: int64